In [4]:
import os
#import PySpin
from IPython import get_ipython
from PIL import Image
import datetime
import matplotlib.pyplot as plt
import sys
#import keyboard
import time
from datetime import datetime
import imageio
import math
import numpy as np
import scipy.signal as sig
from scipy.special import expit, logit
from scipy.stats import norm
import scipy.optimize as optimize
import scipy.fftpack as fftpack
import scipy.ndimage as ndi
import cv2
import warnings
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import mean_squared_error
from scipy.spatial import KDTree
from skimage.transform import hough_line, hough_line_peaks
from scipy.ndimage import sobel
from scipy.ndimage import convolve

In [5]:
def crop_dark_edges(image):
    dark = True
    top = 0
    bottom = -1
    left = 0
    right = -1
    while (dark and top < image.shape[0]):
        line = image[top]
        if (np.average(line) < 80):
            top +=1
        else:
            dark = False
    dark = True
    while (dark and abs(bottom) < image.shape[0]//2):
        line = image[bottom]
        if (np.average(line) < 80):
            bottom -=1
        else:
            dark = False
    dark = True
    while (dark and left < image.shape[1]):
        line = image[:,left]
        if (np.average(line) < 80):
            left += 1
        else:
            dark = False
    dark = True
    while (dark and abs(right)< image.shape[1]//2):
        line = image[:,right]
        if (np.average(line)< 80):
            right -=2
        else:
            dark = False
    return image[top:bottom,left:right]

In [43]:
def horizontal_sobel(image):
    image = image.astype(np.float32)
    sobelled = sobel(image, 0) # apply a horizontal sobel filter: looking for vertical stripes
    #normalize the image
    sobelled = np.abs(sobelled)
    biggest = np.max(sobelled) 
    sobelled = (sobelled/biggest)*255
    return sobelled

In [6]:
# returns a binary image given an image that has been edge filtered and the threshold at which something counts as white
def get_edges(edgeimage, threshold):
    edgeimage = edgeimage > threshold
    edgeimage = edgeimage * 255
    return edgeimage

In [36]:
def find_hough_lines_p(image):
    endpoints = cv2.HoughLinesP(image, 1, np.pi/60, 40, minLineLength=15, maxLineGap=100)
    return endpoints

In [35]:
# use hough transform to get the lines on an image
def find_hough_lines(binaryimage):
    # only look for horizontal lines
    testangles = np.linspace(1.4,1.8, 6, endpoint = True)
    h, theta, d = hough_line(binaryimage, theta=testangles)
    # get the 20 lines with highest intensity
    accumulator, angles, rho = hough_line_peaks(h,theta,d, num_peaks = 20)
    # if no lines are found using num_peaks parameter, use the threshold parameter to try and find lines
    if angles.shape[0] == 0:
        return find_hough_lines_threshold(h, theta, d)
    else:
        return angles, rho

In [17]:
def find_hough_lines_threshold(h, theta, d):
    base_threshold = 0
    accumutor, angles, rho = hough_line_peaks(h, theta, d, threshold = base_threshold)
    num_lines = angles.shape[0]
    # keep decreasing thresholds until less than fifteen lines are found
    while num_lines > 15:
        base_threshold += num_lines/8
        accumulator, angles, rho = hough_line_peaks(h, theta, d, threshold = base_threshold)
        # if the number of lines found suddenly decreases to 0, go back to the old threshold and just return that result regardless of how many lines there were
        if angles.shape[0] == 0:
            base_threshold -= num_lines/8
            accumulator, angles, rho = hough_line_peaks(h, theta, d, threshold = base_threshold)
            return angles, rho
        print(base_threshold)
        num_lines = angles.shape[0]
        #print(num_lines)
    #print("threshold:" + str(base_threshold))
    return angles, rho

In [50]:
def get_crop_from_lines(angles, rho, image):
    if len(rho) < 2:
        print("Not enough lines in the right direction were found")
    #print(final_rho)
    # takes the 2 lines that are closest to the edges, and returns the horizontal indices
    return (np.min(rho), np.max(rho))

In [44]:
def get_crop_from_lines_p(endpoints):
    horizontal_lines = []
    print(horizontal_lines)
    for i in range(endpoints.shape[0]):
        current_line = endpoints[i,0]
        x1 = current_line[0]
        x2 = current_line[2]
        y1 = current_line[1]
        y2 = current_line[3]
        angle = math.atan2(y2 - y1, x2 - x1)
        #print(angle)
        # Check if the line is approximately horizontal (angle within a range)
        if  -0.25 <= angle <= 0.25:
            horizontal_lines.append(current_line)
    horizontal_lines = np.array(horizontal_lines)
    y_values = horizontal_lines[:,1]
    np.append(y_values, horizontal_lines[:,3])
    min_y = np.min(y_values)
    max_y = np.max(y_values)
    return min_y, max_y

In [49]:
# takes in a greyscale image and finds the striped region of interest for dof calculations
# assumes that it is looking for vertical lines
def get_cropped_region_old(image):
    image = crop_dark_edges(image)
    if image.size == 0:
        print("Image too dark. Try increasing exposure time.")
        return
    filtered = horizontal_sobel(image)
    edgethreshold = np.average(filtered) * 6
    edges = get_edges(filtered, edgethreshold)
    angles, rho = find_hough_lines(edges)
    #print (angles)
    #print(rho)
    if angles.shape[0] == 0:
        print("No lines found")
        return
    boundaries = get_crop_from_lines(angles, rho, edges)
    print(boundaries)
    if (boundaries is None):
        print("Could not find proper region from lines")
        return
    # give a little space in the region
    lower = int(0.95 * boundaries[0])
    higher = int(1.05 * boundaries[1])
    croppedimage = image[lower:higher,:]
    return croppedimage

In [45]:
def get_cropped_region(image):
    #image = crop_dark_edges(image)
    if image.size == 0:
        print("Image too dark. Try increasing exposure time.")
        return
    filtered = cv2.Canny(image, 20, 80)
    xy = find_hough_lines_p(filtered)
    #print (angles)
    #print(rho)
    if xy.size==0:
        print('No lines found')
        return
    boundaries = get_crop_from_lines_p(xy)
    print(boundaries)
    if (boundaries is None):
        print("Could not find proper region from lines")
        return
    # give a little space in the region
    lower = int(0.95 * boundaries[0])
    higher = int(1.05 * boundaries[1])
    croppedimage = image[lower:higher,:]
    return croppedimage

In [46]:
def contrast_ratios(image):
    contrast_ratios = []
    for i in range(image.shape[0]):
        row = image[i]
        # calculate max and min pixel value in each row
        row_low = float(np.min(row))#np.quantile(row, 0.1)
        row_high = float(np.max(row))#np.quantile(row, 0.95)
        print("Low " + str(row_low))
        print("High " + str(row_high))      
        # calculte contrast ratio of each row
        contrastratio = (row_high - row_low)/ (2* (row_high+row_low))
        print(contrastratio)
        contrast_ratios.append(contrastratio)
    #normalize contrast ratios
    contrast_ratios = contrast_ratios / np.max(contrast_ratios)
    #fig, ax1 = plt.subplots()
    #ax1.plot(contrast_ratios)
    #ax1.set_ylabel('Contrast Ratios')
    # determine what contrast counts as 'in focus' - this is 0.9 of the maximum
    cutoff = contrast_ratios > 0.9
    indices = np.nonzero(cutoff)
    lower = np.min(indices)
    higher = np.max(indices)
    in_focus_region = image[lower: higher,:]
    # returns the pixel length of the depth of field, the image cropped to only the in focus region of stripes, and an array of all the contrast ratios
    return higher - lower, in_focus_region, contrast_ratios

In [12]:
def calc_dof_len(in_dist, mag, pixel_size):
    return (in_dist * pixel_size) / mag

In [12]:
def configure_exposure(cam):
    """
     This function configures a custom exposure time. Automatic exposure is turned
     off in order to allow for the customization, and then the custom setting is
     applied.
    """

    print('*** CONFIGURING EXPOSURE ***\n')

    try:
        result = True

        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic exposure. Aborting...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off)
        print('Automatic exposure disabled...')

        if cam.ExposureTime.GetAccessMode() != PySpin.RW:
            print('Unable to set exposure time. Aborting...')
            return False

        # Ensure desired exposure time does not exceed the maximum
        exposure_time_to_set = EXPOSURE_TIME
        exposure_time_to_set = min(cam.ExposureTime.GetMax(), exposure_time_to_set)
        cam.ExposureTime.SetValue(exposure_time_to_set)
        print('Shutter time set to %s us...\n' % exposure_time_to_set)

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [13]:
def reset_exposure(cam):
    """
    This function returns the camera to a normal state by re-enabling automatic exposure.
    """
    try:
        result = True

        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to enable automatic exposure (node retrieval). Non-fatal error...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Continuous)

        print('Automatic exposure enabled...')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [14]:
def configure_gain(cam):

    print('*** CONFIGURING GAIN ***\n')

    try:
        result = True

        if cam.GainAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic gain. Aborting...')
            return False

        cam.GainAuto.SetValue(PySpin.GainAuto_Off)
        print('Automatic gain disabled...')

    
        if cam.Gain.GetAccessMode() != PySpin.RW:
            print('Unable to set gain . Aborting...')
            return False

        # Ensure desired exposure time does not exceed the maximum
        gain_to_set = GAIN
        cam.Gain.SetValue(gain_to_set)
        print('Gain set to %s ...\n' % gain_to_set)

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [15]:
def reset_gain(cam):
    try:
        result = True

        if cam.GainAuto.GetAccessMode() != PySpin.RW:
            print('Unable to enable gain exposure (node retrieval). Non-fatal error...')
            return False

        cam.GainAuto.SetValue(PySpin.GainAuto_Continuous)

        print('Automatic gain enabled...')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result 

In [16]:
%matplotlib inline

In [ ]:
continue_recording = True # always True
PLOT_PHASES = False # plot phase vs x in mode 0
PLOT_FITS = False # plot fits in mode 0
PLOT_HIST = False # plot histogram in mode 0
DOME_POINTS = False # plot points at center of image and at dome LED positions
BIN_SIZE = 8 # bin size for binning image in mode 0
PIXEL_SIZE = .0024 # in mm
LINES_MM = 1 # lines/mm for mode 0
EXPOSURE_TIME = 117.0 # in microseconds
CROP_H = 0
GAIN = 0.0
CROP_FACTOR = 1 # < 1 automatically crops image displayed. set to 1 for no crop
CROP_MAG = .2 # relative size of image to calc magnification of 
MAGNIFICATION = 1
now = datetime.now() # current date and time
date_time = now.strftime("%m-%d-%Y_%H-%M-%S")
# mode 2 saves image as both bmp and png, specify names below 
IMAGE_NAME_STRING = '35mm_.1mag_69mmwd_1lpmm_DOF_target'

ill_list = []

%matplotlib qt
plt.ion()

def handle_close(evt):
    """
    This function will close the GUI when close event happens.

    :param evt: Event that occurs when the figure closes.
    :type evt: Event
    """

    global continue_recording
    continue_recording = False


def acquire_and_display_images(cam, nodemap, nodemap_tldevice):
    """
    This function continuously acquires images from a device and display them in a GUI.

    :param cam: Camera to acquire images from.
    :param nodemap: Device nodemap.
    :param nodemap_tldevice: Transport layer device nodemap.
    :type cam: CameraPtr
    :type nodemap: INodeMap
    :type nodemap_tldevice: INodeMap
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    global continue_recording

    sNodemap = cam.GetTLStreamNodeMap()

    # Change bufferhandling mode to NewestOnly
    node_bufferhandling_mode = PySpin.CEnumerationPtr(sNodemap.GetNode('StreamBufferHandlingMode'))
    if not PySpin.IsAvailable(node_bufferhandling_mode) or not PySpin.IsWritable(node_bufferhandling_mode):
        print('Unable to set stream buffer handling mode.. Aborting...')
        return False

    # Retrieve entry node from enumeration node
    node_newestonly = node_bufferhandling_mode.GetEntryByName('NewestOnly')
    if not PySpin.IsAvailable(node_newestonly) or not PySpin.IsReadable(node_newestonly):
        print('Unable to set stream buffer handling mode.. Aborting...')
        return False

    # Retrieve integer value from entry node
    node_newestonly_mode = node_newestonly.GetValue()

    # Set integer value from entry node as new value of enumeration node
    node_bufferhandling_mode.SetIntValue(node_newestonly_mode)
    
    print('set stream buffer handling mode to newest only.')

    print('*** IMAGE ACQUISITION ***\n')
    try:
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        if not PySpin.IsAvailable(node_acquisition_mode) or not PySpin.IsWritable(node_acquisition_mode):
            print('Unable to set acquisition mode to continuous (enum retrieval). Aborting...')
            return False

        # Retrieve entry node from enumeration node
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        if not PySpin.IsAvailable(node_acquisition_mode_continuous) or not PySpin.IsReadable(
                node_acquisition_mode_continuous):
            print('Unable to set acquisition mode to continuous (entry retrieval). Aborting...')
            return False

        # Retrieve integer value from entry node
        acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()

        # Set integer value from entry node as new value of enumeration node
        node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

        print('Acquisition mode set to continuous...')

        # begin acquisition
        cam.BeginAcquisition()

        print('Acquiring images...')

        # retrieve device serial number
        device_serial_number = ''
        node_device_serial_number = PySpin.CStringPtr(nodemap_tldevice.GetNode('DeviceSerialNumber'))
        if PySpin.IsAvailable(node_device_serial_number) and PySpin.IsReadable(node_device_serial_number):
            device_serial_number = node_device_serial_number.GetValue()
            print('Device serial number retrieved as %s...' % device_serial_number)

        # Close program
        print('Press (and hold) enter to close the program...')
        
        i = 0
        
        while(i < 1):
            try:
                dof_array = []

                #  Retrieve next received image
                #
                #  *** NOTES ***
                #  Capturing an image houses images on the camera buffer. Trying
                #  to capture an image that does not exist will hang the camera.
                #
                #  *** LATER ***
                #  Once an image from the buffer is saved and/or no longer
                #  needed, the image must be released in order to keep the
                #  buffer from filling up.
                
                image_result = cam.GetNextImage(1000)

                #  Ensure image completion
                if image_result.IsIncomplete():
                    print('Image incomplete with image status %d ...' % image_result.GetImageStatus())

                else:
                    
                    # Getting the image data as a numpy array
                    image_data = image_result.GetNDArray()  
                    plt.imshow(image_data, cmap= "gray")
                    print(image_data)
                    print(image_data.size)
                    # image_data = crop_center(image_data, math.floor(image_data.shape[1]*CROP_FACTOR), 
                                            # math.floor(image_data.shape[0]*CROP_FACTOR)) # not super sure why this first crop is required 
                    
                    # create a FIGURE inside which all data required and image can be viewed side by side 
                    fig = plt.figure(layout='constrained')
                    
                    # separate the figure into two subplots 
                    ax_arrays = fig.subplots(1, 3, squeeze=False)
                    
                    cropped = get_cropped_region(image_data)
                    if not cropped is None:
                        dof_in_pixels = contrast_ratios(cropped)
                        magnification = MAGNIFICATION
                        dof = calc_dof_length(dof_in_pixels[0], magnification, PIXEL_SIZE)
                        dof_array.append(dof)
                        # plot the in focus region of the current image
                        ax_arrays[0, 0].imshow(dof_in_pixels[1], cmap='gray')
                        # plot contrast ratios for one image
                        ax_arrays[0,1].plot(dof_in_pixels[2])
                        ax_array[0,1].set_ylabel('Contrast ratios')
                        
                        plt.pause(15)
                        plt.close()
                        
                    i += 1
                    plt.plot(dof_array)
                    # If user presses enter, close the program
                    if keyboard.is_pressed('ENTER'):
                        print('Program is closing...')
                        
                        # Close figure
                        plt.close('all')             
                        input('Done! Press Enter to exit...')
                        continue_recording=False                        

                #  Release image
                #
                #  *** NOTES ***
                #  Images retrieved directly from the camera (i.e. non-converted
                #  images) need to be released in order to keep from filling the
                #  buffer.
                image_result.Release()

            except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
                return False

        #  End acquisition
        #
        #  *** NOTES ***
        #  Ending acquisition appropriately helps ensure that devices clean up
        #  properly and do not need to be power-cycled to maintain integrity.
        cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True


def run_single_camera(cam):
    """
    This function acts as the body of the example; please see NodeMapInfo example
    for more in-depth comments on setting up cameras.

    :param cam: Camera to run on.
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True

        nodemap_tldevice = cam.GetTLDeviceNodeMap()

        # Initialize camera
        cam.Init()
        
        nodemap = cam.GetNodeMap()

        if not configure_gain(cam):
            return False
        if not configure_exposure(cam):
            return False
        
        # cam.SensorShutterMode.SetValue(PySpin.SensorShutterMode_Rolling)
        # cam.GammaEnable.SetValue(False)
        # cam.PixelFormat.SetValue(PySpin.PixelFormat_Mono16)
        
        # Retrieve GenICam nodemap        
        # Acquire images
        result &= acquire_and_display_images(cam, nodemap, nodemap_tldevice)
        
        # Reset exposure
        result &= reset_exposure(cam)
        result &= reset_gain(cam)
        
        # Deinitialize camera
        cam.DeInit()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result


def main():
    """
    Example entry point; notice the volume of data that the logging event handler
    prints out on debug despite the fact that very little really happens in this
    example. Because of this, it may be better to have the logger set to lower
    level in order to provide a more concise, focused log.

    :return: True if successful, False otherwise.
    :rtype: bool
    """
    result = True
    # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Get current library version
    version = system.GetLibraryVersion()
    print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()

    num_cameras = cam_list.GetSize()

    print('Number of cameras detected: %d' % num_cameras)

    # Finish if there are no cameras
    if num_cameras == 0:

        # Clear camera list before releasing system
        cam_list.Clear()

        # Release system instance
        system.ReleaseInstance()

        print('Not enough cameras!')
        input('Done! Press Enter to exit...')
        return False

    # Run example on each camera
    for i, cam in enumerate(cam_list):
        print('Running example for camera %d...' % i)

        result &= run_single_camera(cam)
        print('Camera %d example complete... \n' % i)

    # Release reference to camera
    # NOTE: Unlike the C++ examples, we cannot rely on pointer objects being automatically
    # cleaned up when going out of scope.
    # The usage of del is preferred to assigning the variable to None.
    del cam

    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()
    return result

if __name__ == '__main__':
    if main():
        sys.exit(0)
    else:
        sys.exit(1)